In [25]:
import config

HARMONIZE_PIPELINES = "HARMONIZE_PIPELINES"
SET_REFERENCES_PIPELINES = "SET_REFERENCES_PIPELINES"
EXTRA_FIELDS = 'extra_fields'

In [26]:
def getHarmonizer(soort):
    if soort == 'Spoor':
         return  [
            {"$match" : {"table" : {"$in" : ["SPOOR","SPOREN"]}}}, 
            {"$replaceRoot" : {"newRoot" : {"_id" : "$_id", "brondata" : "$$ROOT"}}}, 
            {"$addFields" : { 
                "soort" : "Spoor", 
                "projectcd" : "$brondata.project"
            }}]
    elif soort == 'Put':
         return  [
            {"$match" : {"table" : {"$in" : ["PUT","PUTTEN"]}}}, 
            {"$replaceRoot" : {"newRoot" : {"_id" : "$_id", "brondata" : "$$ROOT"}}}, 
            {"$addFields" : { 
                "soort" : "Put", 
                "projectcd" : "$brondata.project"
            }}]
    else:
        return []
    
lst = [{"Hallo": "Halo"}, getHarmonizer("Spoor")]

In [27]:
lst[1]

[{'$match': {'table': {'$in': ['SPOOR', 'SPOREN']}}},
 {'$replaceRoot': {'newRoot': {'_id': '$_id', 'brondata': '$$ROOT'}}},
 {'$addFields': {'soort': 'Spoor', 'projectcd': '$brondata.project'}}]

In [28]:
isinstance(lst[1], list)

True

In [31]:
wasstraat_model = {
  "Put": {
        HARMONIZE_PIPELINES: [getHarmonizer('Put')],
        SET_REFERENCES_PIPELINES: [[ 
            { '$match': { '$and': [{'putnr': { '$exists': True }}, {'projectcd': { '$exists': True }}]}},
            { '$group':{'_id': {"projectcd" : "$projectcd", 'putnr': "$putnr"}}},
            { '$unwind': "$_id"},
            { '$project': {'_id': 0, 'projectcd': "$_id.projectcd", 'putnr': "$_id.putnr"}},
            { '$addFields': {'key': { '$concat': [ "P", "$projectcd", "P", {'$toString': "$putnr"}] }}},  		
            { '$addFields': {'key_project': { '$concat': [ "P", "$projectcd"]}}},
            { '$addFields': {'soort': 'Put'}}	
            #, { '$merge': {'into': config.COLL_ANALYSE_CLEAN}}
        ]]
  },
  "Vlak": {
        HARMONIZE_PIPELINES: [[]],
        SET_REFERENCES_PIPELINES: [[ 
            { '$match': {'vlaknr': { '$exists': True }}},
            { '$group':{'_id': {"projectcd" : "$projectcd", 'putnr': "$putnr", 'vlaknr': "$vlaknr"}}},
            { '$unwind': "$_id"},
            { '$project': {'_id': 0, 'projectcd': "$_id.projectcd", 'putnr': "$_id.putnr", 'vlaknr': "$_id.vlaknr"}},
            { '$addFields': {'key': { '$concat': [ "P", "$projectcd", {'$ifNull': [{'$concat': ["P", {'$toString': "$putnr" }]}, ""]}, "V", {'$toString': "$vlaknr"}] }}},  		
            { '$addFields': {'key_project': { '$concat': [ "P", "$projectcd"]}}},
            { '$addFields': {'key_put': { '$concat': [ "P", "$projectcd", {'$ifNull': [{'$concat': ["P", {'$toString': "$putnr" }]}, ""]}] }, 'soort': 'Vlak'}}	
            #, { '$merge': {'into': config.COLL_ANALYSE_CLEAN}}
        ]]
  },
  "Spoor": {
        HARMONIZE_PIPELINES: [getHarmonizer('Spoor')],
        SET_REFERENCES_PIPELINES: [[ 
            {'$match': {'spoornr': { '$exists': True }}},
            { '$group':{'_id': {'projectcd':"$projectcd", 'putnr':"$putnr", 'spoornr':"$spoornr", 'vlaknr':"$vlaknr"}, 'aard': {'$max': "$aard"}}},  
            { '$unwind': "$_id"},
            { '$project': {'_id': 0, 'projectcd': "$_id.projectcd", 'putnr': "$_id.putnr", 'spoornr': "$_id.spoornr", 'vlaknr': "$_id.vlaknr"}},
            { '$addFields': {'key': { '$concat': [ "P", "$projectcd", 
                {'$ifNull': [{'$concat': ["P", {'$toString': "$putnr" }]}, ""]},
                {'$ifNull': [{'$concat': ["V", {'$toString': "$vlaknr"}]}, ""]}, "S", {'$toString': "$spoornr"}] }}},  		
            { '$addFields': {'key_vlak': { '$concat': [ "P", "$projectcd", 
                {'$ifNull': [{'$concat': ["P", {'$toString': "$putnr" }]}, ""]},
                {'$ifNull': [{'$concat': ["V", {'$toString': "$vlaknr"}]}, ""]}] }}},  	
            { '$addFields': {'key_project': { '$concat': [ "P", "$projectcd"]}}},
            { '$addFields': {'soort': 'Spoor'}}	
            #, { '$merge': config.COLL_ANALYSE_CLEAN}
        ]]
  }}

In [32]:
def getHarmonizePipeline(soort):
    if soort in wasstraat_model.keys():
        return wasstraat_model[soort][HARMONIZE_PIPELINES][0]
    else:
        raise Exception(f'Fout bij het opvragen van metadata. {soort} is een onbekend metadatasoort.')

def getHarmonizePipelines(soort):
    if soort in wasstraat_model.keys():
        return wasstraat_model[soort][HARMONIZE_PIPELINES]
    else:
        raise Exception(f'Fout bij het opvragen van metadata. {soort} is een onbekend metadatasoort.')


In [35]:
getHarmonizePipeline("Put")

[[{'$match': {'table': {'$in': ['PUT', 'PUTTEN']}}},
  {'$replaceRoot': {'newRoot': {'_id': '$_id', 'brondata': '$$ROOT'}}},
  {'$addFields': {'soort': 'Put', 'projectcd': '$brondata.project'}}]]

In [8]:
d = [{"Hallo": "Hallo"}, {"Hallo2": "Hallo2"}]
d['Hallo']

TypeError: list indices must be integers or slices, not str